In [4]:
import cv2
import numpy as np
import csv
import pandas as pd

In [5]:
def calculate_bluriness(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    lap = cv2.Laplacian(gray, cv2.CV_64F)
    return np.var(lap)

def detect_lighting_change(prev_frame, curr_frame):
    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    curr_gray = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)
    
    diff = cv2.absdiff(prev_gray, curr_gray)
    _, threshold = cv2.threshold(diff, 30, 255, cv2.THRESH_BINARY)
    return cv2.countNonZero(threshold)

def detect_block(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    _, threshold = cv2.threshold(gray, 10, 255, cv2.THRESH_BINARY)
    black_ratio = cv2.countNonZero(threshold) / (frame.shape[0] * frame.shape[1])
    return black_ratio 

def calculate_movement(prev_frame, curr_frame):
    abs_diff = cv2.absdiff(prev_frame, curr_frame)
    movement_score = np.sum(abs_diff) / (curr_frame.shape[0] * curr_frame.shape[1] * 255)
    return movement_score 

In [6]:
videos = ['/Users/elhidze/Downloads/Test/anomaly/1.mp4',
          '/Users/elhidze/Downloads/Test/anomaly/2.mp4',
          '/Users/elhidze/Downloads/Test/anomaly/3.mp4',
          '/Users/elhidze/Downloads/Test/anomaly/4.mp4',
          '/Users/elhidze/Downloads/Test/anomaly/5.mp4',
          '/Users/elhidze/Downloads/Test/anomaly/6.mp4']
csvs = ['/Users/elhidze/Gagarin/csv/test/anomaly/1.csv',
        '/Users/elhidze/Gagarin/csv/test/anomaly/2.csv',
        '/Users/elhidze/Gagarin/csv/test/anomaly/3.csv',
        '/Users/elhidze/Gagarin/csv/test/anomaly/4.csv',
        '/Users/elhidze/Gagarin/csv/test/anomaly/5.csv',
        '/Users/elhidze/Gagarin/csv/test/anomaly/6.csv']

In [7]:
count = 0
for pth in videos:
    print(count)
    cs = csvs[count]
    cap = cv2.VideoCapture(pth)
    
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    csv_file = open(cs, 'w')
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['frame', 'time', 'blurriness', 'lighting_change', 'block', 'movement'])
    
    ret, prev_frame = cap.read()
    
    for frame_number in range(1, frame_count):
        #cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
        
        ret, frame = cap.read()
        if not ret:
            break
        
        time = frame_number / fps
        
        blurriness = calculate_bluriness(frame)
        lighting_change = detect_lighting_change(prev_frame, frame)
        block = detect_block(frame)
        movement = calculate_movement(prev_frame, frame)
        
        csv_writer.writerow([frame_number, time, blurriness, lighting_change, block, movement])
        
        prev_frame = frame
    
    count += 1
    
    csv_file.close()
    cap.release()

0
1


[h264 @ 0x1263b1370] error while decoding MB 101 51, bytestream -5
[h264 @ 0x1263b1370] left block unavailable for requested intra mode
[h264 @ 0x1263b1370] error while decoding MB 0 11, bytestream 26701
[h264 @ 0x1263c3b90] left block unavailable for requested intra mode
[h264 @ 0x1263c3b90] error while decoding MB 0 5, bytestream 22587
[h264 @ 0x1263a8f00] left block unavailable for requested intra mode
[h264 @ 0x1263a8f00] error while decoding MB 0 15, bytestream 190171
[h264 @ 0x1263a8f00] left block unavailable for requested intra mode
[h264 @ 0x1263a8f00] error while decoding MB 0 54, bytestream 19632
[h264 @ 0x1263e8bd0] left block unavailable for requested intra mode
[h264 @ 0x1263e8bd0] error while decoding MB 0 33, bytestream 114094


2


[h264 @ 0x1029a1bf0] error while decoding MB 117 25, bytestream -5
[h264 @ 0x1029987e0] error while decoding MB 65 33, bytestream -9
[h264 @ 0x10297cbb0] left block unavailable for requested intra mode
[h264 @ 0x10297cbb0] error while decoding MB 0 25, bytestream 99428
[h264 @ 0x1029a1bf0] error while decoding MB 93 30, bytestream -7
[h264 @ 0x10297cbb0] left block unavailable for requested intra mode
[h264 @ 0x10297cbb0] error while decoding MB 0 27, bytestream 122458


3


[h264 @ 0x1257db2b0] left block unavailable for requested intra4x4 mode -1
[h264 @ 0x1257db2b0] error while decoding MB 0 26, bytestream 117019


4


[h264 @ 0x1257ee060] error while decoding MB 34 60, bytestream -5


5


In [8]:
anomaly_data = {
    '0': {
        'move': [(48, 52), (93, 98), (140, 142), (180, 185), (248, 249), (313, 315), (413, 414), (471, 474), (521, 522), (615, 619), (670, 674)]
    },
    '2': {
        'blur': [(19, 42), (81, 97), (100, 127), (186, 216), (301, 303), (372, 409), (525, 556), (828, 878), (983, 985), (1014, 1017), (1085, 1087), (1153, 1166)],
        'move': [(141, 142), (240, 247), (751, 756), (758, 762), (919, 924), (983, 985), (1014, 1017), (1085, 1087), (1141, 1142)]
    },
    '3': {
        'blur': [(19, 59), (79, 138)],
        'move': [(0, 19), (40, 60), (80, 106), (121, 141)]
    },
    '4': {
        'block': [(17, 28), (55, 64), (174, 185)],
        'overexposure': [(28, 30), (64, 70)],
        'blur': [(83, 153)]
    },
    '5': {
        'block': [(662, 675), (679, 690)],
        'move': [(3, 4), (324, 325), (454, 455), (507, 508), (508, 560), (547, 662), (714, 718)],
        'blur': [(161, 221)]
    }
  }

anomaly_data_test = {
    '1': {
        'move': [(98, 103), (273, 276), (521, 523), (551, 553)],
        'blur': [(14, 32), (136, 224), (438, 453), (479, 503), (640, 718)],
        'block': [(60, 69), (81, 86), (225, 230), (248, 252), (307, 318), (582, 594), (621, 630), (740, 751)],
        'overexposure': [(69, 71), (230, 233), (252, 255), (594, 598), (630, 634), (751, 754)]
    },
    '2': {
        'block': [(42, 53), (135, 146), (202, 213), (255, 262)],
        'overexposure': [(42, 53), (202, 213)]
    },
    '3': {
        'block': [(125, 131), (249, 258)],
        'overexposure': [(63, 71), (185, 196)]
    },
    '4': {
        'block': [(69, 76), (207, 213)],
        'overexposure': [(133, 140), (257, 265)]
    },
    '5': {
        'block': [(71, 79), (121, 134), (143, 150), (184, 190), (245, 255)],
        'overexposure': [(79, 81), (121, 134), (143, 150), (190, 192), (245, 256)]
    },
    '6': {
        'block': [(63, 69), (182, 192), (241, 260)],
        'overexposure': [(121, 129), (241, 260)]
    }
}

count = 1
for file in csvs:
    df = pd.read_csv(file)
    
    df['normal'] = 1
    df['has_blur'] = 0
    df['has_move'] = 0
    df['has_overexposure'] = 0
    df['has_block'] = 0
    
    for anomaly_type, time_ranges in anomaly_data_test[str(count)].items():
        for start, end in time_ranges:
            df.loc[(df['time'] >= start) & (df['time'] <= end), f'has_{anomaly_type}'] = 1
    
    df.loc[(df['has_blur'] == 0) & (df['has_move'] == 0) & (df['has_overexposure'] == 0) & (df['has_block'] == 0), 'normal'] = 1
    df.loc[(df['has_blur'] == 1) | (df['has_move'] == 1) | (df['has_overexposure'] == 1) | (df['has_block'] == 1), 'normal'] = 0
    
    df.to_csv(file, index=False)
    
    count += 1
    